# Gauss Karışım Modelleri, (GKM) (Gaussian Mixture Models, GMM)
Hazırlayan: Marc Deisenroth
Çeviren: Utku Karaca

- Gauss karışım modelleri ile yoğunluk modellemesi

Gauss karışım modellerinde verinin yoğunluğunu 
$$
p(\boldsymbol x) = \sum_{k=1}^K \pi_k \mathcal{N}(\boldsymbol x|\boldsymbol \mu_k, \boldsymbol \Sigma_k)\,,\quad \pi_k \geq 0\,,\quad \sum_{k=1}^K\pi_k = 1
$$
ile gösteririz. 

$\pi_k$: Karışım ağırlıkları.

Bu uygulamada GKM'leri daha iyi anlamaya çalışacağız. GKM modellerinin Beklenti Maksimizasyonu (BM) algoritması ile eğitimini kodlayacağız. 

Gerekli paketler:

In [ ]:
# nümerik işlemler kütüphanesi
import numpy as np
# görselleştirme kütüphanesi
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc
# veri üretimi
from scipy.stats import multivariate_normal
# yakınsama koşulu
import scipy.linalg as la

%matplotlib inline
# aynı sonuçların elde edilmesi
np.random.seed(42)

### GKM tanımlanması
Gerçek GKM'nin tanımlanması ve veri üretilmesi.

In [ ]:
# 3 bileşenli GKM parametrelerinin oluşturulması

# ortalamalar
o = np.zeros((3,2))
o[0] = np.array([1.2, 0.4])
o[1] = np.array([-4.4, 1.0])
o[2] = np.array([4.1, -0.3])

# kovaryanslar
k = np.zeros((3,2,2))
k[0] = np.array([[0.8, -0.4], [-0.4, 1.0]])
k[1] = np.array([[1.2, -0.8], [-0.8, 1.0]])
k[2] = np.array([[1.2, 0.6], [0.6, 3.0]])

# karışım ağırlıkları
w = np.array([0.3, 0.2, 0.5])

Veri yaratımı:

In [ ]:
N_bilesen = 200 # her bir bileşenden gelecek veri sayısı
N = N_bilesen*3 # toplam veri sayısı
x = []
y = []
for i in range(3):
    x_tmp, y_tmp = np.random.multivariate_normal(o[i], 
                                                 k[i], 
                                                 N_bilesen).T 
    x = np.hstack([x, x_tmp])
    y = np.hstack([y, y_tmp])

veri = np.vstack([x, y])

Verinin görselleştirilmesi:

In [ ]:
X, Y = np.meshgrid(np.linspace(-10,10,100), 
                   np.linspace(-10,10,100))
pos = np.dstack((X, Y))

# veri kümesinin görselleştirilmesi
plt.figure()
plt.title("Karışım bileşenleri")
plt.plot(x, y, 'ko', alpha=0.3)
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

# dağılımların ortalamalarının görsele eklenmesi
plt.plot(o[:,0], o[:,1], 'or')

# olasılık dağılım fonksiyon konturlarının grafiğe eklenmesi
for j in range(3):
    mvn = multivariate_normal(o[j,:].ravel(), k[j,:,:])
    xx = mvn.pdf(pos)
    plt.contour(X, Y, xx,  alpha = 1.0, zorder=10)
     
# GKM görselleştirilmesi
plt.figure()
plt.title("GKM")
plt.plot(x, y, 'ko', alpha=0.3)
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

# ağırlıklarla birlikte GKM oluşturulması
gkm = 0
for j in range(3):
    mix_comp = multivariate_normal(o[j,:].ravel(), k[j,:,:])
    gkm += w[j]*mix_comp.pdf(pos)
    
plt.contour(X, Y, gkm,  alpha = 1.0, zorder=10);

## BM Algoritması ile GKM eğitimi
### BM Parametrelerinin yaratılması

In [ ]:
K = 3 # küme sayısı

ortalamalar = np.zeros((K,2))
kovaryanslar = np.zeros((K,2,2))

for k in range(K):
    ortalamalar[k] = np.random.normal(size=(2,))
    kovaryanslar[k] = np.eye(2)

agirliklar = np.ones((K,1))/K
print("Başlangıç ortalama vektörleri (bir vektör/satır):\n" + str(ortalamalar))

In [ ]:
NLL = [] # GKM'nin negatif logaritmik olabilirliği
gkm_nll = 0

for k in range(K):
    gkm_nll += agirliklar[k]*multivariate_normal.pdf(mean=ortalamalar[k,:], 
                                                     cov=kovaryanslar[k,:,:], 
                                                     x=veri.T)
NLL += [-np.sum(np.log(gkm_nll))]
print('Negatif olabilirlik:',NLL)

plt.figure()
plt.plot(x, y, 'ko', alpha=0.3)
plt.plot(ortalamalar[:,0], ortalamalar[:,1], 'oy', markersize=25)

for k in range(K):
    rv = multivariate_normal(ortalamalar[k,:], kovaryanslar[k,:,:])
    plt.contour(X, Y, rv.pdf(pos), alpha = 1.0, zorder=10)
    
plt.xlabel("$x_1$");
plt.ylabel("$x_2$");

BM algoritmasının B-adımında model parametrelerine $\pi_k, \boldsymbol\mu_k, \boldsymbol\Sigma_k$ göre güncellenecek olan değerlerin tanımlanması: 
\begin{align*}
    r_{nk} &:= \frac{\pi_k\mathcal N(\boldsymbol
          x_n|\boldsymbol\mu_k,\boldsymbol\Sigma_k)}{\sum_{j=1}^K\pi_j\mathcal N(\boldsymbol
          x_n|\boldsymbol \mu_j,\boldsymbol\Sigma_j)} &(11.17)
\end{align*}
B-adımında güncellenen değerler ile model parametrelerinin güncellenmesi (M-adımı)
\begin{align*}
\boldsymbol\mu_k^\text{new} &= \frac{1}{N_k}\sum_{n = 1}^Nr_{nk}\boldsymbol x_n\,, & (11.20)\\
   \boldsymbol\Sigma_k^\text{new}&= \frac{1}{N_k}\sum_{n=1}^Nr_{nk}(\boldsymbol x_n-\boldsymbol\mu_k)(\boldsymbol x_n-\boldsymbol\mu_k)^\top\,,& (11.30)\\
   \pi_k^\text{new} &= \frac{N_k}{N}, & (11.42)
\end{align*}

$N_k$ tanımı:
$$
N_k := \sum_{n=1}^N r_{nk}
$$

## BM Algoritması

In [ ]:
r = np.zeros((K,N))

for em_iter in range(100):    
    # B-adımı: r'lerin güncellenmesi
    for k in range(K):
        r[k] = agirliklar[k]*multivariate_normal.pdf(mean=ortalamalar[k,:],
                                                     cov=kovaryanslar[k,:,:], x=veri.T)  
    r = r/np.sum(r, axis=0) 
    # M-adımı
    N_k = np.sum(r, axis=1)
    for k in range(K): 
        # ortalamaların güncellenmesi
        ortalamalar[k] = np.sum(r[k]*veri, axis=1)/N_k[k]
        # kovaryansların güncellenmesi
        diff = veri - ortalamalar[k:k+1].T
        _tmp = np.sqrt(r[k:k+1])*diff
        kovaryanslar[k] = np.inner(_tmp, _tmp)/N_k[k]
    # ağırlıklar
    agirliklar = N_k/N 
    
    # log-olabilirlik (log-likelihood)
    gkm_nll = 0
    for k in range(K):
        gkm_nll += agirliklar[k]*multivariate_normal.pdf(mean=ortalamalar[k,:].ravel(),
                                                         cov=kovaryanslar[k,:,:], x=veri.T)
    NLL += [-np.sum(np.log(gkm_nll))]
    
    plt.figure() 
    plt.plot(x, y, 'ko', alpha=0.3)
    plt.plot(ortalamalar[:,0], ortalamalar[:,1], 'oy', markersize=25)
    for k in range(K):
        rv = multivariate_normal(ortalamalar[k,:], kovaryanslar[k])
        plt.contour(X, Y, rv.pdf(pos), alpha = 1.0, zorder=10)
        
    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    plt.text(x=3.5, y=8, s="EM iterasyon "+str(em_iter+1))
    
    if la.norm(NLL[em_iter+1]-NLL[em_iter]) < 1e-6:
        print(str(em_iter+1)+". iterasyondan sonra yakınsadı.")
        break

In [ ]:
# son karışım modelinin görseli
plt.figure() 
gkm = 0
for k in range(3):
    bilesen_karisik = multivariate_normal(ortalamalar[k,:].ravel(), 
                                          kovaryanslar[k,:,:])
    gkm += agirliklar[k]*bilesen_karisik.pdf(pos)

plt.plot(x, y, 'ko', alpha=0.3)
plt.contour(X, Y, gkm,  alpha = 1.0, zorder=10)    
plt.xlim([-8,8]);
plt.ylim([-6,6]);

print(ortalamalar)
print(o)

In [ ]:
plt.figure()
plt.semilogy(np.linspace(1,len(NLL), len(NLL)), NLL)
plt.xlabel("BM iterasyon");
plt.ylabel("Negatif Log-Olabilirlik (Negative log-likelihood)");

idx = [0, 1, 9, em_iter+1]

for i in idx:
    plt.plot(i+1, NLL[i], 'or')